# initialize

In [2]:
pip install selenium

     |████████████████████████████████| 904 kB 485 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13487 sha256=b628a531ef9e83fc18cc98cfffa990f071abbcd6bc12919be4788d3f4b2c9e9e
  Stored in directory: /Users/Rebeca/Library/Caches/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
import json
from fake_useragent import UserAgent

In [53]:
import re

# 股票字典
* 可以在後台進行設定
* 不用每一次輸入都建立一次字典

In [2]:
### 股票轉代碼
res = requests.get("https://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
df = pd.read_html(res.text)[0]
# 設定column名稱
df.columns = df.iloc[0]
# 刪除第一行
df = df.iloc[1:]
df = df.iloc[1:]
stock = {}  # 股票中文對代碼的字典
for i in range(2, len(df)):
    try:
        stock['%s' % df['有價證券代號及名稱'][i].split()[1]] = df['有價證券代號及名稱'][i].split()[0]
    except:
        break
stockinverse = {}  # 股票代碼對中文的字典
for i in range(2, len(df)):
    try:
        stockinverse['%s' % df['有價證券代號及名稱'][i].split()[0]] = df['有價證券代號及名稱'][i].split()[1]
    except:
        break



In [3]:
df.head()

,有價證券代號及名稱,國際證券辨識號碼(ISIN Code),上市日,市場別,產業別,CFICode,備註
2,1101 台泥,TW0001101004,1962/02/09,上市,水泥工業,ESVUFR,NaN
3,1102 亞泥,TW0001102002,1962/06/08,上市,水泥工業,ESVUFR,NaN
4,1103 嘉泥,TW0001103000,1969/11/14,上市,水泥工業,ESVUFR,NaN
5,1104 環泥,TW0001104008,1971/02/01,上市,水泥工業,ESVUFR,NaN
6,1108 幸福,TW0001108009,1990/06/06,上市,水泥工業,ESVUFR,NaN


In [ ]:
df

In [27]:
stockinverse#['0050']

{'1101': '台泥',
 '1102': '亞泥',
 '1103': '嘉泥',
 '1104': '環泥',
 '1108': '幸福',
 '1109': '信大',
 '1110': '東泥',
 '1201': '味全',
 '1203': '味王',
 '1210': '大成',
 '1213': '大飲',
 '1215': '卜蜂',
 '1216': '統一',
 '1217': '愛之味',
 '1218': '泰山',
 '1219': '福壽',
 '1220': '台榮',
 '1225': '福懋油',
 '1227': '佳格',
 '1229': '聯華',
 '1231': '聯華食',
 '1232': '大統益',
 '1233': '天仁',
 '1234': '黑松',
 '1235': '興泰',
 '1236': '宏亞',
 '1256': '鮮活果汁-KY',
 '1301': '台塑',
 '1303': '南亞',
 '1304': '台聚',
 '1305': '華夏',
 '1307': '三芳',
 '1308': '亞聚',
 '1309': '台達化',
 '1310': '台苯',
 '1312': '國喬',
 '1313': '聯成',
 '1314': '中石化',
 '1315': '達新',
 '1316': '上曜',
 '1319': '東陽',
 '1321': '大洋',
 '1323': '永裕',
 '1324': '地球',
 '1325': '恆大',
 '1326': '台化',
 '1337': '再生-KY',
 '1338': '廣華-KY',
 '1339': '昭輝',
 '1340': '勝悅-KY',
 '1341': '富林-KY',
 '1402': '遠東新',
 '1409': '新纖',
 '1410': '南染',
 '1413': '宏洲',
 '1414': '東和',
 '1416': '廣豐',
 '1417': '嘉裕',
 '1418': '東華',
 '1419': '新紡',
 '1423': '利華',
 '1432': '大魯閣',
 '1434': '福懋',
 '1435': '中福',
 '1436': '華友聯',

## 匯入成交金額前20大股票

In [66]:
f = open(r'成交金額前20大.txt')
#f.readlines()

In [65]:
f.close

<function TextIOWrapper.close()>

In [67]:
stock_num=[]
for line in f:
    s=re.findall(r"\d+",line) #\d 匹配任意数字
    for i in s:
        stock_num.append(i)
print(stock_num)   

['2330', '2454', '3661', '3105', '6531', '4968', '2458', '2337', '6488', '6415', '2408', '4966', '6510', '2379', '5274', '5269', '6462', '2303', '3016', '8086']


# 提供查詢股票

In [3]:
### 使用者輸入keyword
# keyword = input('請輸入輸入股票代號或名稱： ') 
for keyword in stock_num:
    ### FakeUser
    # 每次input都丟一個假的user_agent
    def set_header_user_agent():
        user_agent = UserAgent()
        return user_agent.random
    headers = {"user-agent": set_header_user_agent()
               ,"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
               ,"accept-language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7"
              }
    url = 'https://goodinfo.tw/StockInfo/StockDetail.asp'  # 網域：台灣股市資訊網

    # 判斷keyword是否為數字、是數字又是否是股票代碼
    #if ord(keyword[0]) >= 48 and ord(keyword[0]) <= 57:  # 如果使用者輸入的第一個字落在48到57之間，則為數字
    count = 0 # 如果count=0找無此keyword
    
    try:
        print('程式如瘋狗般尋找著：'+keyword)#stockinverse[keyword])
        count = 1  # 如果count = 1則keyword輸入為股票代碼
    except KeyError:
        print('程式如瘋狗般尋找著：'+keyword)#+'('+stock[keyword]+')')
        count = 2  # 如果count = 2則keyword輸入為股票代碼
    if count == 1:
        re = requests.post(url+"?STOCK_ID="+str(keyword), headers = headers)
    elif count == 2:
        re = requests.post(url+"?STOCK_ID="+keyword, headers = headers)

    re.encoding = 'utf-8'  # 設定為charset的編碼格式
    soup = BeautifulSoup(re.text, 'html.parser')  # 利用beautifulsoup解析
    table = soup.find('table', attrs={'class':'solid_1_padding_3_1_tbl'})  # 觀察css code擷取我們需要的表格資訊
    newdf = pd.read_html(str(table))  # 每一個股票代碼會有一個新資訊，並用dataframe形式儲存

    print('久等了，以下是 '+keyword+' 的搜尋結果：')
    if count == 1:
        print('股票名稱：' + keyword)#stockinverse[keyword])
    else:
        print('股票名稱：' + keyword)
    print()
    print('當前該股基本資料：')
    display(newdf[0][1:7])
    print('其他資訊：')
    print(newdf[0][0][7].replace('\xa0', ''))
    # except:
    #     print('不要騙我書讀得少，真的有這隻股票？')
    #     print('更換目標重新輸入吧？')

請輸入輸入股票代號或名稱： 0050


Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/Users/Rebeca/opt/anaconda3/lib/python3.7/site-packages/fake_useragent/utils.py", line 67, in get
    context=context,
  File "/Users/Rebeca/opt/anaconda3/lib/python3.7/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/Users/Rebeca/opt/anaconda3/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/Users/Rebeca/opt/anaconda3/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/Users/Rebeca/opt/anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/Users/Rebeca/opt/anaconda3/lib/python3.7/urllib/request.py", line 1347, in http_open
    return self.do_open(http.client.HTTPConnection, req)
  File "/Users/Rebeca/opt/anaconda3/lib/python3.7/urllib/request.py", line 1322, in d

程式如瘋狗般尋找著：0050
久等了，以下是 0050 的搜尋結果：
股票名稱：0050

當前該股基本資料：


,0,1,2,3,4,5,6,7
1,成交價,昨收,漲跌價,漲跌幅,振幅,開盤,最高,最低
2,89.5,88.35,+1.15,+1.3%,1.13%,88.9,89.8,88.8
3,成交張數,成交金額,成交筆數,成交均張,成交均價,PBR,PER,PEG
4,15550,13.89 億,7617,2 張/筆,89.31 元,NaN,NaN,NaN
5,昨日張數,昨日金額,昨日筆數,昨日均張,昨日均價,昨漲跌價 (幅),昨漲跌價 (幅),昨漲跌價 (幅)
6,9917,8.73 億,5501,1.8 張/筆,88.08 元,+0.75 (+0.86%),+0.75 (+0.86%),+0.75 (+0.86%)


其他資訊：
連漲連跌:連7漲 (+5.65元/+6.74%) 上市指數: 11610.32(130.92/+1.14%)


## 儲存每股股票資訊

In [74]:
def set_header_user_agent():
        user_agent = UserAgent()
        return user_agent.random
error=[]
for keyword in stock_num:
    headers = {"user-agent": set_header_user_agent()
               ,"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
               ,"accept-language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7"
              }
    url = 'https://goodinfo.tw/StockInfo/StockDetail.asp'  # 網域：台灣股市資訊網
    count = 0
    try:
        re = requests.post(url+"?STOCK_ID="+str(keyword), headers = headers)
        re.encoding = 'utf-8'  # 設定為charset的編碼格式
        soup = BeautifulSoup(re.text, 'html.parser')  # 利用beautifulsoup解析
        table = soup.find('table', attrs={'class':'solid_1_padding_3_1_tbl'})  # 觀察css code擷取我們需要的表格資訊
        newdf = pd.read_html(str(table))  # 每一個股票代碼會有一個新資訊，並用dataframe形式儲存

        From = [keyword]*5
        to=[newdf[0][0][2],newdf[0][0][4],newdf[0][5][2],newdf[0][6][2],newdf[0][7][2]]
        relation = ['price','volume','opening','highest price','lowest price']
        dic = {'From':From,'to':to,'relation':relation}
        df = pd.DataFrame(dic,columns=dic.keys())

        df.to_csv('/Users/Rebeca/Fintech_中信專案/stock_info/%s.csv' %keyword)
    except:
        print(url+"?STOCK_ID="+str(keyword))
        error.append(keyword)

https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=2303
https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=3016


## error處理

In [82]:
re = requests.post('https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=3016', headers = headers)
re.encoding = 'utf-8'  # 設定為charset的編碼格式
soup = BeautifulSoup(re.text, 'html.parser')  # 利用beautifulsoup解析
table = soup.find('table', attrs={'class':'solid_1_padding_3_1_tbl'}) 

In [83]:
newdf = pd.read_html(str(table))

In [84]:
to=[newdf[0][0][2],newdf[0][0][4],newdf[0][5][2],newdf[0][6][2],newdf[0][7][2]]
to

['75.3', '7416', '76.3', '77.2', '75.3']

In [85]:
From = ['3016']*5
relation = ['price','volume','opening','highest price','lowest price']
dic = {'From':From,'to':to,'relation':relation}
df = pd.DataFrame(dic,columns=dic.keys())
df.to_csv('/Users/Rebeca/Fintech_中信專案/stock_info/3016.csv')

# 將資訊儲存成表格

In [23]:
From = [keyword]*5
to=[newdf[0][0][2],newdf[0][0][4],newdf[0][5][2],newdf[0][6][2],newdf[0][7][2]]
relation = ['price','volume','opening','highest price','lowest price']

In [26]:
dic = {'From':From,'to':to,'relation':relation}

In [27]:
df3 = pd.DataFrame(dic,columns=dic.keys())

In [28]:
df3

,From,to,relation
0,0050,89.5,price
1,0050,15550,volume
2,0050,88.9,opening
3,0050,89.8,highest price
4,0050,88.8,lowest price


In [89]:
ss = pd.read_csv('/Users/Rebeca/Fintech_中信專案/stock_info/3016.csv',index_col=0)

In [91]:
ss1 = pd.read_csv('/Users/Rebeca/Fintech_中信專案/stock_info/4968.csv',index_col=0)

In [99]:
df = ss1.append(pd.read_csv('/Users/Rebeca/Fintech_中信專案/stock_info/2337.csv',index_col=0))
df.reset_index(drop=True, inplace=True)

In [100]:
df

,From,to,relation
0,4968,173.50,price
1,4968,7668.00,volume
2,4968,180.50,opening
3,4968,183.50,highest price
4,4968,173.00,lowest price
5,2337,32.85,price
6,2337,52391.00,volume
7,2337,33.45,opening
8,2337,33.65,highest price
9,2337,32.75,lowest price
